In [ ]:
import ast
from glob import glob
import sys
import os
from copy import deepcopy

import networkx as nx

from stdlib_list import stdlib_list
STDLIB = set(stdlib_list())

CONVERSIONS = {
    'attr': 'attrs',
    'PIL': 'Pillow',
    'Image': 'Pillow',
    'mpl_toolkits': 'matplotlib',
    'dateutil': 'python-dateutil'
}

In [ ]:
dirtree = nx.DiGraph()

exclude_dirs = {'node_modules', '__pycache__', 'dist'}
exclude_files = {'__init__.py', '_version.py', '_install_requires.py'}
packages_dir = os.path.join(ROOT, 'packages')

for root, dirs, files in os.walk(packages_dir, topdown=True):
    dirs[:] = [d for d in dirs if d not in exclude_dirs]
    
    if '__init__.py' in files:
        module_init = os.path.join(root, '__init__.py')
        files[:] = [f for f in files if f not in exclude_files]
        
        dirtree.add_node(module_init)
        parent_init = os.path.join(os.path.dirname(root), '__init__.py')
        if os.path.exists(parent_init):
            dirtree.add_edge(parent_init, module_init)

        for f in files:
            if f.endswith('.py'):
                filepath = os.path.join(root, f)
                dirtree.add_node(filepath)
                dirtree.add_edge(module_init, filepath)

In [ ]:
package_roots = [n for n, d in dirtree.in_degree() if d == 0]
package_root_map = {
    os.path.basename(os.path.dirname(package_root)): package_root
    for package_root in package_roots
}

internal_packages = list(package_root_map.keys())
internal_packages

In [ ]:
import_types = {
    type(ast.parse('import george').body[0]),
    type(ast.parse('import george as macdonald').body[0])}

import_from_types = {
    type(ast.parse('from george import macdonald').body[0])
}

all_import_types = import_types.union(import_from_types)

all_import_types

In [ ]:
def get_imports(filepath):
    with open(filepath, 'r') as file:
        data = file.read()

    parsed = ast.parse(data)
    imports = [node for node in ast.walk(parsed) if type(node) in all_import_types]

    stdlib_imports = set()
    external_imports = set()
    internal_imports = set()
    near_relative_imports = set()
    far_relative_imports = set()
    
    def get_base_converted_module(name):
        name = name.split('.')[0]
        
        try:
            name = CONVERSIONS[name]
        except KeyError:
            pass
        
        return name
    
    def add_level_0(name):
        if name in STDLIB:
            stdlib_imports.add(name)
        elif name in internal_packages:
            internal_imports.add(name)
        else:
            external_imports.add(name)

    for an_import in imports:
        
        if type(an_import) in import_types:
            for alias in an_import.names:
                name = get_base_converted_module(alias.name)
                add_level_0(name)
                
        elif type(an_import) in import_from_types:
            name = get_base_converted_module(an_import.module)
            if an_import.level == 0:
                add_level_0(name)
            elif an_import.level == 1:
                near_relative_imports.add(name)
            else:
                far_relative_imports.add(name)
                
        else:
            raise
            
    
            
    return {
        'stdlib': stdlib_imports,
        'external': external_imports,
        'internal': internal_imports,
        'near_relative': near_relative_imports,
        'far_relative': far_relative_imports}

In [ ]:
all_imports = {
    filepath: get_imports(filepath)
    for filepath in dirtree.nodes()
}

In [ ]:
def get_descendants_dependencies(filepath):
    dependencies = deepcopy(all_imports[filepath])
    
    for descendant in nx.descendants(dirtree, filepath):
        for key, item in all_imports[descendant].items():
            dependencies[key] |= item
            
    return dependencies

In [ ]:
package_dependencies = {
    package: get_descendants_dependencies(root)
    for package, root in package_root_map.items()
}

package_dependencies

In [ ]:
get_descendants_dependencies(package_roots[0])

In [ ]:
list(nx.neighbors(dirtree, package_roots[4]))

In [ ]:
nx.descendants(dirtree, '/home/simon/git/pymedphys/packages/pymedphys/src/pymedphys/__init__.py')

In [ ]:
# nx.neighbors()

In [ ]:
imports = [node for node in ast.walk(table) if type(node) in all_import_types]
imports

In [ ]:
# external_imports = set()
# near_internal_imports = set()
# far_internal_imports = set()

# for an_import in imports:
#     if type(an_import) in import_types:
#         for alias in an_import.names:
#             external_imports.add(alias.name)
#     elif type(an_import) in import_from_types:
#         if an_import.level == 0:
#             external_imports.add(an_import.module)
#         elif an_import.level == 1:
#             near_internal_imports.add(an_import.module)
#         else:
#             far_internal_imports.add(an_import.module)
#     else:
#         raise
        
#     print(ast.dump(an_import))

In [ ]:
external_imports

In [ ]:
near_internal_imports

In [ ]:
far_internal_imports